In [ ]:
!pip install openai yt-dlp

In [ ]:
# OPENAI KEY lesen
import os
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get('OPENAI_KEY')
except:
    OPENAI_KEY = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


In [ ]:
from pathlib import Path
from openai import OpenAI
import yt_dlp
from IPython.display import Audio, display

In [ ]:
client = OpenAI()

# Speech-to-Text - Audio Dateien transkribieren

## Donwload Youtube-Video + extrahiere Audiospur

https://www.youtube.com/watch?v=EwJFSZE2kws

In [ ]:
def download_audio(link):
  with yt_dlp.YoutubeDL({'extract_audio': True, 'format': 'bestaudio', 'outtmpl': '%(title)s.mp3'}) as video:
    info_dict = video.extract_info(link, download = True)
    video_title = info_dict['title']
    video.download(link)    
    return f'{video_title}.mp3'

file_name = download_audio('https://www.youtube.com/watch?v=EwJFSZE2kws')

## "Simple" Transkription

In [ ]:
audio_file= open(file_name, "rb")
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)
print(transcription.text)

## Transkription mit Zeitstempeln

https://platform.openai.com/docs/guides/speech-to-text/timestamps

In [ ]:
transcript = client.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word"]
)

print(transcript.words[:100])

# Text-to-Speech - Texte vorlesen lassen

In [ ]:
speech_file_path = Path('text-to-speech.mp3')
response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input=transcription.text
)

response.stream_to_file(speech_file_path)

In [ ]:
display(Audio(speech_file_path, autoplay=True))